In [1]:
import pandas as pd 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import *
from tensorflow.keras.layers import *

df = pd.read_csv(
    './aws_a5_1_all.csv'
)

df.head()

,num,cputime_dif,depth,high,image,tension,time,velocity,volume,weld_velocity,width,wire_l
0,0,0.109213,-2.612515,2.251549,4482.0,20.0,2.614,5.5,0.704238,8.0,4.850269,15.0
1,1,0.113643,-2.568461,2.261362,4497.0,20.0,2.629,5.5,0.093257,8.0,5.014591,15.0
2,2,0.187733,-2.284407,2.307767,5361.0,20.0,3.493,5.5,5.371661,8.0,6.496406,15.0
3,3,0.124918,-2.072830,2.291701,5433.0,20.0,3.565,5.5,0.447639,8.0,7.534782,15.0
4,4,0.110063,-2.497762,2.195335,6325.0,20.0,4.457,5.5,5.545741,8.0,7.738949,15.0


In [7]:
labels = np.array(df[['high' , 'tension' , 'velocity' , 'volume' , 'weld_velocity' , 'width' , 'wire_l']])
features = np.array(df[['depth']])

labels_mean = np.mean(labels , axis=0)
labels_std = np.std(labels , axis = 0)

features

array([[-2.61251488],
       [-2.56846062],
       [-2.28440721],
       ...,
       [-3.09060336],
       [-3.09060336],
       [-3.09058904]])

In [8]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

for row in numeric_dataset.take(3):
    print(row)

(<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-2.61251488])>, <tf.Tensor: shape=(7,), dtype=float64, numpy=
array([ 2.25154901, 20.        ,  5.5       ,  0.7042383 ,  8.        ,
        4.85026926, 15.        ])>)
(<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-2.56846062])>, <tf.Tensor: shape=(7,), dtype=float64, numpy=
array([ 2.26136213, 20.        ,  5.5       ,  0.09325717,  8.        ,
        5.01459088, 15.        ])>)
(<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-2.28440721])>, <tf.Tensor: shape=(7,), dtype=float64, numpy=
array([ 2.30776663, 20.        ,  5.5       ,  5.37166119,  8.        ,
        6.49640607, 15.        ])>)


In [9]:
mean = np.mean(features , axis=0)
mean

array([-2.57594384])

In [12]:
standard = np.std(features , axis= 0)
standard

array([0.64405404])

In [13]:
def normalize(data , label) :
    return (data - mean) / standard , (label - labels_mean) / labels_std

x_test = numeric_dataset.take(10)

numeric_batches = numeric_dataset.shuffle(1000).batch(4)
numeric_batches = numeric_batches.map(normalize)

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='tanh'),
      tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(15, activation='tanh'),
      tf.keras.layers.Dense(10, activation='tanh'),
      tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dense(7 , activation='tanh')
  ])

  model.compile(optimizer= optimizers.Adam() ,
                loss= losses.mean_squared_error ,
                metrics= [metrics.MeanSquaredError()])
  return model

model = get_basic_model()

model.fit(numeric_batches , batch_size=4 , epochs=30)

Epoch 1/30

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

252/252 [==============================] - 3s 12ms/step - loss: 0.9099 - mean_squared_error: 0.9099
Epoch 2/30
252/252 [==============================] - 3s 12ms/step - loss: 0.8930 - mean_squared_error: 0.8930
Epoch 3/30
252/252 [==============================] - 3s 11ms/step - loss: 0.8801 - mean_squared_error: 0.8801
Epoch 4/30
252/252 [==============================] - 3s 11ms/step - loss: 0.8596 - mean_squared_error: 0.8596
Epoch 5/30
252/252 [==============================] - 3s 12ms/st

In [26]:
test = (np.array([[-2.612515]]) - mean) / standard
print(test)
model.predict(test)*labels_std + labels_mean

[[-0.05678275]]


array([[ 1.80694465, 27.53100325,  7.25691962,  0.79824874, 11.18717043,
         6.58996991, 13.33239386]])

In [27]:
#important
model.save('./my_model2')

INFO:tensorflow:Assets written to: ./my_model2\assets


In [28]:
new_model = tf.keras.models.load_model('./my_model2')

test = (np.array([[-2.612515]]) - mean) / standard
print(test)
model.predict(test)*labels_std + labels_mean

[[-0.05678275]]


array([[ 1.80694465, 27.53100325,  7.25691962,  0.79824874, 11.18717043,
         6.58996991, 13.33239386]])